In [5]:
# test_models.py
from pydantic import BaseModel, Field
from typing import List, Optional
from datetime import datetime
import uuid
import json

# ==========================================
# 1. The Core Data Contracts (MVP)
# ==========================================

class RecommendationRequest(BaseModel):
    query: str
    top_k: int = 5  # Default to 5 results
    
    # Optional: Keep these flexible for future "Advanced Search"
    # usage: filters={"sector": "Technology"}
    filters: Optional[dict] = None 

class StockRecommendation(BaseModel):
    # Identity
    ticker: str
    name: str
    
    # The "Why" (RAG Output)
    why_fits: str
    
    # Context (MVP UI needs)
    current_price: float
    sector: str

class RecommendationResponse(BaseModel):
    recommendations: List[StockRecommendation]
    query_id: str = Field(default_factory=lambda: str(uuid.uuid4()))
    processing_time_ms: float = 0.0

# ==========================================
# 2. Test Functions
# ==========================================

def example_request():
    """Example of creating a clean MVP request"""
    
    # 1. The simplest possible request (what the user actually types)
    simple_req = RecommendationRequest(
        query="I want cheap EV stocks",
        top_k=3
    )
    
    print("--- Request Payload (Frontend -> Backend) ---")
    print(simple_req.model_dump_json(indent=2))
    return simple_req

def example_response(request: RecommendationRequest):
    """Example of the flattened, simple response"""
    
    # Simulate a "Hit" found by your RAG pipeline
    rag_results = [
        StockRecommendation(
            ticker="F",
            name="Ford Motor Company",
            why_fits="Ford is aggressively pivoting to EVs with the F-150 Lightning and trades at a lower valuation than pure-play EV stocks.",
            current_price=12.05,
            sector="Consumer Cyclical"
        ),
        StockRecommendation(
            ticker="GM",
            name="General Motors",
            why_fits="GM has committed $35B to EV and AV development and offers a 'cheap' entry point compared to Tesla.",
            current_price=39.50,
            sector="Consumer Cyclical"
        )
    ]
    
    # Wrap it in the standard response envelope
    response = RecommendationResponse(
        recommendations=rag_results,
        processing_time_ms=450.5  # Example: "Found in 0.45s"
    )
    
    print("\n--- Response Payload (Backend -> Frontend) ---")
    print(response.model_dump_json(indent=2))
    return response

def validate_mvp_rules():
    """Ensures we don't break the 'Bare Metal' rules"""
    
    print("\n--- Validation Checks ---")
    
    # Rule: current_price must be a number, not a string
    try:
        bad_price = StockRecommendation(
            ticker="TSLA", name="Tesla", why_fits="...", sector="Tech",
            current_price="$100"  # <--- STRING ERROR
        )
    except Exception as e:
        print(f"✓ Correctly caught string price error: {str(e).split('type')[0]}...")

    # Rule: Missing fields (like 'sector') should fail
    try:
        missing_sector = StockRecommendation(
            ticker="TSLA", name="Tesla", why_fits="...", current_price=100.0
            # <--- MISSING SECTOR
        )
    except Exception as e:
        print(f"✓ Correctly caught missing field: {str(e).split('Field')[0]}...")

# ==========================================
# 3. Execution
# ==========================================

if __name__ == "__main__":
    req = example_request()
    res = example_response(req)
    validate_mvp_rules()

--- Request Payload (Frontend -> Backend) ---
{
  "query": "I want cheap EV stocks",
  "top_k": 3,
  "filters": null
}

--- Response Payload (Backend -> Frontend) ---
{
  "recommendations": [
    {
      "ticker": "F",
      "name": "Ford Motor Company",
      "why_fits": "Ford is aggressively pivoting to EVs with the F-150 Lightning and trades at a lower valuation than pure-play EV stocks.",
      "current_price": 12.05,
      "sector": "Consumer Cyclical"
    },
    {
      "ticker": "GM",
      "name": "General Motors",
      "why_fits": "GM has committed $35B to EV and AV development and offers a 'cheap' entry point compared to Tesla.",
      "current_price": 39.5,
      "sector": "Consumer Cyclical"
    }
  ],
  "query_id": "2a08f84d-3081-450e-8f1e-ef63885fef23",
  "processing_time_ms": 450.5
}

--- Validation Checks ---
✓ Correctly caught string price error: 1 validation error for StockRecommendation
current_price
  Input should be a valid number, unable to parse string as a numb